In [1]:

import numpy as np
import random as rd
from numba import njit, jit
import sys, os
from setup import SHORT_PATH
import importlib.util
game_name = 'Phom'

def add_game_to_syspath(game_name):
    if len(sys.argv) >= 2:
        sys.argv = [sys.argv[0]]
    sys.argv.append(game_name)

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHORT_PATH}Base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

add_game_to_syspath(game_name)
env = setup_game(game_name)

getActionSize = env.getActionSize
getStateSize = env.getStateSize
getAgentSize = env.getAgentSize

getValidActions = env.getValidActions
getReward = env.getReward
numba_main_2 = env.numba_main_2

@njit()
def bot_lv0(state, perData):
    validActions = getValidActions(state)
    arr_action = np.where(validActions==1)[0]
    idx = np.random.randint(0, arr_action.shape[0])
    return arr_action[idx], perData


In [4]:
@njit()
def getValueCard(state):
  cards = state[0:51]
  cards = np.where(cards == 1)[0]
  cardInPhom = state[104+52+52:104+52+52+52]
  cardInPhom = np.where(cardInPhom == 1)[0]
  arr_return = np.zeros(len(cards))
  for i in range(len(cards)):
     for j in range(i,len(cards)):
       if cards[i] // 4 == cards[j] // 4:
          arr_return[i] += 1
       if (cards[j] // 4) - (cards[i] // 4) == 1 and cards[j] % 4 == cards[i] % 4 :
         arr_return[i] += 1
     cost_ =  cards[i] * 0.07
     arr_return[i] -= cost_
     if cards[i] in cardInPhom:
        arr_return[i] = 100000
  return arr_return
@njit()
def agentPhom(state,per):
    actions = getValidActions(state)
    actions = np.where(actions == 1)[0]
    cardInPhom = state[104+52+52:104+52+52+52]
    cardInPhom = np.where(cardInPhom == 1)[0]
    cards = state[0:51]
    cards = np.where(cards == 1)[0]
    if 0 in actions:    
        return 0,per
    if 1 in actions:
        return 1,per
    arr_value = getValueCard(state)
    idx_ = np.argmin(arr_value)
    if  cards[idx_] + 2 in actions:
        return cards[idx_] + 2,per
    return np.random.choice(actions),per
win,per = numba_main_2(agentPhom,10000,np.array([0.]),1)
print(win)

6917
